## Lesson 6: Code extension by adding a new segment in the ELF

**Objectives**: add a new segment in an ELF; use the OFRAK PatchMaker to convert a C patch into a binary patch, including a linking step

## Section Overview
In this section, we will replace a call to `puts` with a wrapper function that converts all lower-case characters to upper-case in the target x86 ELF `hello_world`. We will explore what happens when and why patches can be applied in various ways and then learn about OFRAK PatchMaker in order to implement the objective.

In [1]:
### Target binary `hello_world`
from ofrak_tutorial.helper_functions import create_hello_world_binary

# Hello world source code for reference
HELLO_WORLD_SOURCE = r"""
#include <stdio.h>
int main() {
   // Can you patch `puts` to capitalize all lower-case characters in a provided string?
   puts("Hello, World!\n");
   return 0;
}
"""

create_hello_world_binary()

### Patch source `uppercase_and_print`
c_patch = r"""
extern int puts(char *str); // This is the address of what Ghidra calls 'puts@GLIBC'

void uppercase_and_print(char *text)
{
    // We can't modify the string directly since it's stored in a RO segment
    // so we need to copy it and provide the modified string to puts.

    char str[15] = {0};
    for(int i=0; i<14; i++){
        if(text[i] >= 0x61 && text[i] <= 0x7A)
            str[i] = text[i]-0x20;
        else
            str[i] = text[i];
    }
    puts(str);
}
"""

c_patch_filename = "c_patch.c"
with open(c_patch_filename, "w") as f:
    f.write(c_patch)

We shall allocate in the free-space of `hello_world` the wrapper function `uppercase_and_print`, which is called instead of `puts` in `main`. `uppercase_and_print` will copy the string from RO memory, modify it, then call the library function `extern int puts(char *str)` with the modified string buffer.

The final result of the patch, once disassembled with Ghidra, should look like this:
```
                             **************************************************************
                             *                          FUNCTION                          *
                             **************************************************************
                             undefined main()
             undefined         AL:1           <RETURN>
                             main                                            XREF[5]:     Entry Point(*),
                                                                                          _start:0040105d(*),
                                                                                          _start:0040105d(*), 00402038,
                                                                                          004020d8(*)
        00401122 55              PUSH       RBP
        00401123 48 89 e5        MOV        RBP,RSP
        00401126 48 8d 3d        LEA        RDI,[s_Hello,_World!_00402004]                   = "Hello, World!"
                 d7 0e 00 00
        0040112d e8 ce 3e        CALL       FUN_00405000                                     undefined FUN_00405000()
                 00 00
        00401132 b8 00 00        MOV        EAX,0x0
                 00 00
        00401137 5d              POP        RBP
        00401138 c3              RET

```

In both the original and patched binaries, the address of "Hello, World!\n" is loaded from the read-only segment containing `.rodata`. In the patched binary, the call to `puts` has been replaced with a call to a function Ghidra identifies as `FUN_00405000`. `FUN_00405000` is our compiled function `uppercase_and_print`, which when disassembled looks like this:

```
                             //
                             // segment_10
                             // Loadable segment  [0x405000 - 0x406fff] (disabled execute
                             // ram:00405000-ram:00406fff
                             //
                             **************************************************************
                             *                          FUNCTION                          *
                             **************************************************************
                             undefined FUN_00405000()
             undefined         AL:1           <RETURN>
             undefined8        Stack[-0x9]:8  local_9                                 XREF[1]:     00405004(RW)
             undefined1        Stack[-0x10]:1 local_10                                XREF[2]:     0040500a(RW),
                                                                                                   00405036(*)
                             FUN_00405000                                    XREF[2]:     00400280(*), main:0040112d(c)
        00405000 48 83 ec 18     SUB        RSP,0x18
        00405004 48 83 64        AND        qword ptr [RSP + local_9],0x0
                 24 0f 00
        0040500a 48 83 64        AND        qword ptr [RSP + local_10],0x0
                 24 08 00
        00405010 6a f2           PUSH       -0xe
        00405012 58              POP        RAX
                             LAB_00405013                                    XREF[1]:     00405034(j)
        00405013 48 85 c0        TEST       RAX,RAX
        00405016 74 1e           JZ         LAB_00405036
        00405018 0f b6 4c        MOVZX      ECX,byte ptr [RDI + RAX*0x1 + 0xe]
                 07 0e
        0040501d 8d 51 9f        LEA        EDX,[RCX + -0x61]
        00405020 8d 71 e0        LEA        ESI,[RCX + -0x20]
        00405023 80 fa 1a        CMP        DL,0x1a
        00405026 40 0f b6 d6     MOVZX      EDX,SIL
        0040502a 0f 43 d1        CMOVNC     EDX,ECX
        0040502d 88 54 04 16     MOV        byte ptr [RSP + RAX*0x1 + 0x16],DL
        00405031 48 ff c0        INC        RAX
        00405034 eb dd           JMP        LAB_00405013
                             LAB_00405036                                    XREF[1]:     00405016(j)
        00405036 48 8d 7c        LEA        RDI=>local_10,[RSP + 0x8]
                 24 08
        0040503b e8 f0 bf        CALL       <EXTERNAL>::puts                                 int puts(char * __s)
                 ff ff
        00405040 48 83 c4 18     ADD        RSP,0x18
        00405044 c3              RET

```

## Strategies of Patching
#### Function Wrapping
**Wrapping** is relatively straightforward as it typically only requires allocating the new code in free space and changing the destination address of a function call to point to that new code. This is the strategy taken for this tutorial section.

Normally wrapping introduces an extra indirection to the call-chain for the function being wrapped. The original `puts` we're replacing is already a thunk that itself jumps to the extern function `puts@GLIBC`; since the `puts` function we're replacing branches to `puts@GLIBC` anyway, one may erroneously assume that this particular case doesn't introduce additional indirection.

But in our patch we implement a /call/ to the thunk `puts` from within `uppercase_and_print`, whereas the original `puts` thunk /jumps/ to `puts@GLIBC` as part of a tail-call, skipping stack frame allocation operations that a call would imply.
```
Original / unaltered `hello_world` x86 ELF `puts` callgraph

 ________  CALL   ________  JUMP ______________
|  main  |  -->  |  puts  |  +  |  puts@GLIBC  |
|________|       |________|     |______________|
      ^_________________________________|
        TAIL-RET TO PC AFTER `puts` CALL
```
```
Patched `hello_world` x86 ELF `puts` callgraph

 ________  CALL   _______________________  CALL   ________   JUMP ______________
|  main  |  -->  |  uppercase_and_print  |  -->  |  puts  |   +  |  puts@GLIBC  |
|________|       |___aka FUN_00405000____|       |________|      |______________|
      ^__________________ ^ _____|                                       |
      :                   |______________________________________________|
      :                            TAIL-RET TO PC AFTER `puts` CALL
      :
      RET TO PC AFTER `uppercase_and_print` CALL
```
In our case, it isn't guaranteed that the argument provided to `uppercase_and_print` would be writeable, so we have to copy it first to writeable memory, capitalize all the characters, and call `puts@GLIBC` with a pointer to the modified buffer. Because we are allocating a new buffer, we may as well allocate an extra stack frame for it between the `main` parent and the `puts@GLIBC` call; any other solution would require relocating the string or making the segment read-writeable.

#### Function replacement
Alternatively the calling function could be **replaced** to capitalize all characters in the string before passing it to `puts`. In our case, we'd replace `main` with our patch. This would require deciding on how much behavior to replicate of the original caller into the patched caller, but it also allows for us to deal with more complex transformations and non-composable modifications on programs, and keeping these modifications inlined.

Similar to how we do it for function wrapping, the patched caller is allocated into free-space, then the binary containing that patch is re-linked to use the patched caller in place of the original.

Since we know the string that we're replacing, we can allocate it from RW memory and capitalize its letters in-place instead.

```
Caller-replaced `hello_world` x86 ELF `puts` callgraph

int alternative_main() {
    // Initialize the string in modifyable memory
    char text[] = { "Hello, World!\n" };

    // Capitalize in-place
    for (int i = 0; i < sizeof text; i++) {
        if(text[i] >= 0x61 && text[i] <= 0x7A)
            text[i] = text[i] - 0x20;
    }

    // Print
    puts(text);

    return 0;
}

 ____________________  CALL   ________  JUMP ______________
|  alternative_main  |  -->  |  puts  |  +  |  puts@GLIBC  |
|____________________|       |________|     |______________|
                  ^_________________________________|
                    TAIL-RET TO PC AFTER `puts` CALL
```

#### Data-only patching
Generally if the data is provided in RO memory, we could just replace the string with a capitalized variant, instead of modifying the program to capitalize the string at runtime. [Lesson 1](./1_simple_string_modification.ipynb) addresses data-only patching. For this lesson we will perform a code patch for demonstration purposes.


#### Considering other places to patch
Could we hook something else to capitalize strings without copying from read-only memory? We could look for an existing function that already performs an array copy to writeable memory, and replace it with a modified version that also capitalizes each character as they are written to the destination. Such a function does not exist in our `hello_world` example as the string is directly passed to `puts` from RO memory. However,  we may consider patching GLIBC instead. If you expand the call-chain for GLIBC `puts` then it should look something like this:

```
Printing a string to STDOUT, from `main` to the `write` syscall:

 ________         ________         ______________         ____________         _______________________
|  main  |  -->  |  puts  |   +   |  puts@GLIBC  |  -->  |  _IO_puts  |  -->  |  _IO_new_file_xsputn  |
|________|       |________|       |______________|       |____________|       |_______________________|
                                                                                   CALL |   CALL |
                                                                                       \|/       |
                                                                                     ___`___     |
                                                                                   [ mempcpy ]   |
                                                                                                 |
                                                                                                \|/
                              _________________         ______________________        ______`_________
                             |  syscall write  |  <--  |  _IO_new_file_write  |  <-- |  new_do_write  |
                             |_________________|       |______________________|      |________________|

```



It looks like [_IO_new_file_xsputn](https://elixir.bootlin.com/glibc/glibc-2.29/source/libio/fileops.c#L1180) [line 1243](https://elixir.bootlin.com/glibc/glibc-2.29/source/libio/fileops.c#L1243) is a macro expanded into a memcpy. One could replace the [mempcpy](https://elixir.bootlin.com/glibc/glibc-2.36/source/sysdeps/i386/i686/mempcpy.S#L37) call with something that performs a `BYTE != 0x20 ? BYTE & ~ 0x20 : BYTE` operation before writing each byte to the destination. The modified `mempcpy` would be embedded in the free-space of the target binary and be called in `__mempcpy`'s place on [line 1243](https://elixir.bootlin.com/glibc/glibc-2.29/source/libio/fileops.c#L1243).

While this patch would mitigate the introduction of an extra indirection by replacing (not wrapping) an existing `memcpy`, there are two major drawbacks to this strategy: additional machinery would need to be implemented to prevent the modified `mempcpy` from modifying bytes un-intended for STDOUT, and this patch depends on specific versions of GLIBC to be loaded into the target binary's runtime. This patch seems more complicated to implement properly and is less portable due to its dependence on system-provided runtime libraries; nonetheless, whether it is suitable is dependent on the constraints a patch author is working with.

**Overall**, a patch author has to wear two hats simultaneously: a patch author needs to apply the mindset of an engineer to decide on designs that fit some set of constraints, while also applying the mindset of a scientist to model behavior when modifications are made (and possibly on a binary without available source code). Certain patches may favor modularity and portability while other patch designs may favor performance, resulting in less portable and lower-level optimizations being applied; and in either case there may be requirements on timing and state guarantees that the modification has to satisfy.

## Using OFRAK PatchMaker

In order to apply this patch we'll need to correctly compile and link the wrapper function `uppercase_and_print` into the `hello_world` ELF binary. We'll use OFRAK's *PatchMaker* to facilitate building and injecting the patch.

We can break this task down into 6 steps:

1. **Unpack** `hello_world` onto an OFRAK resource tree;
2. **Extend** `hello_world` with a new ELF segment to hold the code for `uppercase_and_print`, by applying OFRAK _LiefAddSegmentModifier_ to the resource tree;
3. **Patch** the call instruction to `puts` (in main) with `uppercase_and_print` using OFRAK core; then finally,
4. **Compile** the `uppercase_and_print` patch source using OFRAK _PatchMaker_, after defining the program attributes, toolchain and symbols we wish to re-link to `hello_world`;
5. **Inject** the extended ELF segment with the compiled patch blob using OFRAK _BinaryPatchModifier_;
6. **Pack** the OFRAK resource we've modified back into an executable, so we can run and test our work!


#### [1] Unpack `hello_world` onto an OFRAK resource tree

First lets instantiate OFRAK with the Ghidra analyzer backend and create a root resource from the `hello_world` binary we want to patch:

In [2]:
import ofrak_ghidra
from ofrak import OFRAK

ofrak = OFRAK()
ofrak.discover(ofrak_ghidra)
binary_analysis_context = await ofrak.create_ofrak_context()

root_resource = await binary_analysis_context.create_root_resource_from_file("hello_world")

Using OFRAK Community License.


#### [2] Extend the `hello_world` resource with a new ELF segment with OFRAK core

With the root resource created, let's start with the creation of the new segment. We will use OFRAK's `LiefAddSegmentModifier`, leveraging the [LIEF project](https://lief-project.github.io/) to add an ELF segment which will contain the code for `uppercase_and_print`.

In [3]:
from ofrak import ResourceFilter
from ofrak.core import LiefAddSegmentConfig, LiefAddSegmentModifier, ElfProgramHeader

PAGE_ALIGN = 0x1000


async def add_and_return_segment(elf_resource, vaddr, size):
    """Add a segment to `elf_resource`, of size `size` at virtual address `vaddr`,
    returning this new segment resource after unpacking."""

    config = LiefAddSegmentConfig(vaddr, PAGE_ALIGN, [0 for _ in range(size)], "rx")
    await elf_resource.run(LiefAddSegmentModifier, config)

    await elf_resource.unpack_recursively()

    # Get our newly added segment. First get all ElfProgramHeaders, then return the one
    # with our virtual address.
    file_segments = await elf_resource.get_descendants_as_view(
        ElfProgramHeader, r_filter=ResourceFilter(tags=(ElfProgramHeader,))
    )
    return [seg for seg in file_segments if seg.p_vaddr == vaddr].pop()


new_segment = await add_and_return_segment(root_resource, 0x405000, 0x2000)

print(f"new segment: {new_segment}")

openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Debian-1deb11u1)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Debian-1deb11u1, mixed mode)
openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Debian-1deb11u1)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Debian-1deb11u1, mixed mode)


new segment: ElfProgramHeader(segment_index=10, p_type=1, p_offset=16384, p_vaddr=4214784, p_paddr=4214784, p_filesz=8192, p_memsz=8192, p_flags=5, p_align=4096)


#### [3] Patch the call instruction to `puts` (in main) with `uppercase_and_print` using OFRAK core

Let's also get the instruction patch to call `uppercase_and_print` (which will start where our new segment starts) instead of `puts` out of the way:

In [4]:
from ofrak.core import ComplexBlock, Instruction
from ofrak import ResourceAttributeValueFilter


async def call_new_segment_instead(root_resource, new_segment):
    """Replace the original `call` instruction in main with a call to the start of `new_segment`."""
    main_cb = await root_resource.get_only_descendant_as_view(
        v_type=ComplexBlock,
        r_filter=ResourceFilter(
            attribute_filters=(ResourceAttributeValueFilter(ComplexBlock.Symbol, "main"),)
        ),
    )
    call_instruction = await main_cb.resource.get_only_descendant_as_view(
        v_type=Instruction,
        r_filter=ResourceFilter(
            attribute_filters=(ResourceAttributeValueFilter(Instruction.Mnemonic, "call"),)
        ),
    )
    await call_instruction.modify_assembly("call", f"0x{new_segment.p_vaddr:x}")


await call_new_segment_instead(root_resource, new_segment)

#### [4] **Compile** the `uppercase_and_print` patch source using _PatchMaker_

The previous steps we've prepared `hello_world` to fit `uppercase_and_print` by modifying its structure in the resource tree. In this step we shall finally use _PatchMaker_ to compile the patch, which we shall break down into the following sub-steps:

- Define the `ProgramAttributes` dataclass, which specifies the target CPU, ISA, etc.;
- Define the `ToolchainConfig` dataclass, which specifies toolchain configuration parameters, such as optimization flags, reloc flags, etc.;
- Initialize `PatchMaker` with the toolchain variant to use (LLVM, GCC, vbcc and version) and the symbol addresses to re-link, which in the case for `hello_world` is `puts`;
- Define the `BOM` (Batch of Objects and Metadata) dataclass to include the source file with the `uppercase_and_print` patch;
- Define the `PatchRegionConfig` dataclass describing the object files of the patch; then finally,
- Compile the patch into a `FEM` (Final Executable and Metadata) object

We can then use the FEM to extract the segment containing `uppercase_and_print` and re-import the contents of the patched segment into the OFRAK tree.

In [5]:
import tempfile
import os

bld_dir = tempfile.mkdtemp()
c_patch_filename = "c_patch.c"
exec_path = os.path.join(bld_dir, "hello_world_patch_exec")
input_filename = "hello_world"
output_filename = "HELLO_WORLD"


`hello_world` is a 64-bit x86 ELF binary compiled with GCC, however in this example we will compile the patch with LLVM. It works, but mixing objects compiled by different toolchains and compiler flags could increase the chances of un-predictable failures in other examples. Usually the most predictable results (and failures) are achieved by matching the toolchain configuration of the host binary, which can be inferred through some reverse engineering (fingerprinting, strings, debug info).

Let's create the `ProgramAttributes` with information we know about the target binary:

In [6]:
from ofrak.core import ProgramAttributes
from ofrak_type.architecture import InstructionSet
from ofrak_type.bit_width import BitWidth
from ofrak_type.endianness import Endianness

proc = ProgramAttributes(
    isa=InstructionSet.X86,
    sub_isa=None,
    bit_width=BitWidth.BIT_64,
    endianness=Endianness.BIG_ENDIAN,
    processor=None,
)

In supporting multiple toolchains, `ToolchainConfig` specifies generic compiler features (optimization level, force inline functions, etc.) which _PatchMaker_ maps into specific compiler flags provided to each toolchain. A full description can be found in the docstring for `ToolchainConfig` in `ofrak_patch_maker/ofrak_patch_maker/toolchain/model.py`. Let's compile our patch with `-OS` optimization, function inlining, as well as reduce extraneous scaffolding that may be added by the compiler:

In [7]:
from ofrak_patch_maker.toolchain.model import (
    ToolchainConfig,
    BinFileType,
    CompilerOptimizationLevel,
)

tc_config = ToolchainConfig(
    file_format=BinFileType.ELF,
    force_inlines=True,
    relocatable=False,
    no_std_lib=True,
    no_jump_tables=True,
    no_bss_section=True,
    create_map_files=True,
    compiler_optimization_level=CompilerOptimizationLevel.SPACE,
    debug_info=False,
    check_overlap=False,
)

Having specified `ProgramAttributes` and `ToolchainConfig`, we can set up the `PatchMaker` to link `extern int puts(char *str)` in our patch to the original address of `puts` in our target binary:

In [8]:
import logging

from ofrak_patch_maker.patch_maker import PatchMaker
from ofrak_patch_maker.toolchain.llvm_12 import LLVM_12_0_1_Toolchain

# Get the complex block containing the code for `puts`
puts_cb = await root_resource.get_only_descendant_as_view(
    v_type=ComplexBlock,
    r_filter=ResourceFilter(
        attribute_filters=(ResourceAttributeValueFilter(ComplexBlock.Symbol, "puts"),)
    ),
)

# Initialize the PatchMaker. This is where we tell it that our `_puts` will
# need to be linked to the address of the existing `puts`.
toolchain = LLVM_12_0_1_Toolchain(proc, tc_config)
patch_maker = PatchMaker(
    toolchain=toolchain,
    logger=logging.getLogger("ToolchainTest"),
    build_dir=bld_dir,
    base_symbols={
        "puts": puts_cb.virtual_address,
    },
)

print(
    f"Once the BOM is created, `extern int puts(char *str)` will point to 0x{puts_cb.virtual_address:x}, the address of the `puts` thunk in `hello_world` which can be accessed from within the patch source code."
)

Once the BOM is created, `extern int puts(char *str)` will point to 0x401030, the address of the `puts` thunk in `hello_world` which can be accessed from within the patch source code.


_PatchMaker_ can build a patch from multiple source, object and header files, so we ask it to generate a `BOM` (Batched Objects and Metadata):

In [9]:
# The BOM basically corresponds to the step of building the object files, before linking,
# and gives us more fine-grained control of the build step if we wish to.
bom = patch_maker.make_bom(
    name="hello_world_patch",
    source_list=[c_patch_filename],
    object_list=[],
    header_dirs=[],
)

print(
    f"BOM dataclass:\n"
    f"name:                          {bom.name}\n"
    f"bss_size_required:             {bom.bss_size_required}\n"
    f"entry_point_symbol (optional): {bom.entry_point_symbol}\n\n"
    f"object map[c_patch]:           {bom.object_map['c_patch.c']}\n"
)

BOM dataclass:
name:                          hello_world_patch
bss_size_required:             None
entry_point_symbol (optional): None

object map[c_patch]:           AssembledObject(path='/tmp/tmp10mi8gpg/hello_world_patch_bom_files/c_patch.c.o', file_format=<BinFileType.ELF: 'elf'>, segment_map=immutabledict({'.text': Segment(segment_name='.text', vm_address=0, offset=64, is_entry=False, length=70, access_perms=<MemoryPermissions.RX: 5>, is_allocated=True, is_bss=False, alignment=4)}), strong_symbols=immutabledict({'c_patch.c': (0, <LinkableSymbolType.UNDEF: -1>), 'uppercase_and_print': (0, <LinkableSymbolType.FUNC: 0>)}), unresolved_symbols=immutabledict({'puts': (0, <LinkableSymbolType.UNDEF: -1>)}), bss_size_required=None)



Then we generate the `PatchRegionConfig`, which describes the segment we've extended in step [3] and maps it to the `uppercase_and_print` code blob that we are about to compile:

In [10]:
from ofrak_patch_maker.model import PatchRegionConfig
from ofrak_patch_maker.toolchain.model import Segment
from ofrak_type.memory_permissions import MemoryPermissions

# Tell the PatchMaker about the segment we added in the binary...
text_segment_uppercase = Segment(
    segment_name=".text",
    vm_address=new_segment.p_vaddr,
    offset=0,
    is_entry=False,
    length=new_segment.p_filesz,
    access_perms=MemoryPermissions.RX,
)

# ... And that we want to put the compiled C patch there.
uppercase_object = bom.object_map[c_patch_filename]
segment_dict = {
    uppercase_object.path: (text_segment_uppercase,),
}

# Generate a PatchRegionConfig incorporating the previous information
p = PatchRegionConfig(bom.name + "_patch", segment_dict)

And finally for this step, we compile the patch into an FEM (Final Executable and Metadata) and write it to disk:

In [11]:
from ofrak_patch_maker.toolchain.utils import get_file_format

fem = patch_maker.make_fem([(bom, p)], exec_path)

assert os.path.exists(exec_path)
assert get_file_format(exec_path) == tc_config.file_format

ld.lld: warning: cannot find entry symbol _start; not setting start address


#### [5] Inject the extended ELF segment with the compiled patch blob using OFRAK _BinaryPatchModifier_
Having compiled the patch, we shall extract `uppercase_and_print` from the FEM into the extended segment we created in step [3] on the OFRAK resource tree:

In [12]:
from ofrak.core import BinaryPatchModifier, BinaryPatchConfig

# At this point, the PatchMaker has produced an executable containing our new segment.
# Let's read it, find the binary data of the segment, and finally patch that binary
# data into our resource.
with open(fem.executable.path, "rb") as f:
    exe_data = f.read()

# Retrieve the binary data of our new segment
segment_data = b""
for segment in fem.executable.segments:
    if segment.length == 0 or segment.vm_address == 0:
        continue
    segment_data = exe_data[segment.offset : segment.offset + segment.length]
    break
assert len(segment_data) != 0

# Patch the compiled code in the new_segment
patch_config = BinaryPatchConfig(new_segment.p_offset, segment_data)
await root_resource.run(BinaryPatchModifier, patch_config)

print("OFRAK tree patched!")

OFRAK tree patched!


#### [6] Pack the OFRAK resource we've modified back into an executable, so we can run and test our work!

In [13]:
await root_resource.pack()
await root_resource.flush_data_to_disk(output_filename)

Let's test our results!

In [14]:
%%bash

printf "Original: "
./hello_world

printf "Modified: "
chmod +x ./HELLO_WORLD && ./HELLO_WORLD

Original: Hello, World!
Modified: HELLO, WORLD!


If you can see the capitalized "HELLO, WORLD!" output, the patch has successfully been applied. Feel free to experiment with the patch source and toolchain configuration in this notebook.

[Next page](7_conclusion.ipynb)